In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from math import sqrt, atan, sin , cos
import pyautogui as gui
import pyperclip
from tqdm.notebook import tqdm

Definir Archivo SVG de Entrada:

In [ ]:
PATH_SVG = "C:\\Users\\XAVIER\\3D Objects\\abadia\\abadia.roblox.svg"
LAYER_SVG = "layer1"

- Poner pantallas 50/50 con Roblox Studio
- Ejecutar el siguiente bloque
- En los siguientes 5 segundos situar el cursor en la linea de comando de roblox

In [ ]:
# Posiciones referenciales de iconos
# obtener desde consola usando
# import pyautogui
# pyautogui.displayMousePosition()
print("Esperando Posicion ...")
for x in range(5,0,-1):
    print(x, end=" . ")
    sleep(1)
print("\nSe procede a tomar posición.")

POS_CONSOLA = gui.position()
print(f"Consola de Roblox Studio definida en {POS_CONSOLA}")

In [ ]:
# PLANTILLA_COMANDO = """do local function createPart(name, size, position, color, orientation, material, parent, transparency) local p = Instance.new("Part"); p.Name = name; p.Size = size; p.Position = position; p.BrickColor = BrickColor.new(color); p.Material = material or Enum.Material.Plastic; p.Parent = parent or workspace; p.Anchored = true; p.Transparency = transparency or 0; p.Orientation = orientation or Vector3.new(0, 0, 0); return p end; createPart("{nombre}", Vector3.new({ancho}, {espesor}, {alto}), Vector3.new({x}, {z}, {y}), Vector3.new(0, {angulo_z}, 0) ,"Dark stone grey", Enum.Material.SmoothPlastic ) end """
PLANTILLA_COMANDO = """local p = Instance.new("Part"); p.Name = "{nombre}"; p.Size = Vector3.new( {ancho}, {espesor}, {alto}) ; p.Position = Vector3.new( {x}, {z}, {y}) ; p.BrickColor = BrickColor.new("Dark stone grey"); p.Material = Enum.Material.Plastic; p.Parent = workspace; p.Anchored = true; p.Transparency = 0; p.Orientation = Vector3.new(0, {angulo_z}, 0)"""

def crear_caja( x, y, z, ancho, alto, espesor, angulo_z, nombre="Pared"):
    
    gui.click( POS_CONSOLA[0], POS_CONSOLA[1] )
    sleep(0.5)

    # Borra script anterior
    gui.hotkey('ctrl', 'a')
    sleep(0.5)
    
    mensaje = PLANTILLA_COMANDO.format( nombre = nombre, x = x, y = y, z = z, ancho = ancho, alto = alto, espesor = espesor, angulo_z = angulo_z )
    pyperclip.copy(mensaje)
    gui.hotkey('ctrl', 'v')
    
    sleep(0.5)
    
    gui.press('enter')
    
    sleep(0.5)

    return True

Prueba de Creación de Caja

In [ ]:
crear_caja( x = 25, y = 0, z = 0, ancho = 25, alto = 30, espesor = 5, angulo_z = 45 )

In [ ]:
a = np.array([1,2,3])
print(a)

In [ ]:
def importar_svg(nombre_capa = 'layer7', path_archivo = 'ChullaQuito.svg'):
    archivo = open( path_archivo, 'rt')    
    sopa = BeautifulSoup(archivo,'xml')
    archivo.close()
    capa = sopa.svg.find('g', id = nombre_capa)
    rectangulos = capa.find_all('rect')
    valores = [ [ x['x'], x['y'], x['width'], x['height'], x['transform'] if 'transform' in x.attrs else 'rotate(0)'  ] for x in rectangulos ]
    valores = [ [ float(x[0]), float(x[1]), float(x[2]), float(x[3]), float(x[4][7:-1]) ] for x in valores ]
    return np.array(valores)

def obtener_viewbox(path_archivo):
    archivo = open( path_archivo, 'rt')    
    sopa = BeautifulSoup(archivo,'xml')
    archivo.close()
    viewBox = [float(x) for x in sopa.svg["viewBox"].split()]
    return viewBox
    

In [ ]:
coordenadas1 = importar_svg( nombre_capa = LAYER_SVG, path_archivo = PATH_SVG )
coordenadas1

In [ ]:
viewbox1 = obtener_viewbox(PATH_SVG)
viewbox1

In [ ]:
def adaptar_coordenadas( vector, viewbox = [0.0, 0.0, 256.0, 256.0], escala = 2.0 / 5 ):
    # return [ vector[0] + vector[2] / 2, 256.0 - vector[1] - vector[3] / 2, vector[2], vector[3], - vector[4]  ]
    radio = sqrt( (vector[0] + vector[2] / 2) ** 2 + (vector[1] + vector[3] / 2 ) ** 2 )
    angulo_original_grados = atan( (vector[1] + vector[3] / 2 ) / (vector[0] + vector[2] / 2) ) * 180 / np.pi
    # angulo_original_grados = atan( (vector[1] + vector[3] / 2 ) / radio ) * 180 / np.pi
    nuevo_angulo_rads = (angulo_original_grados + vector[4]) * np.pi / 180
    vector_0 = abs(radio * cos(nuevo_angulo_rads))
    vector_1 = abs(radio * sin(nuevo_angulo_rads))
    traslacion_x = viewbox[0] - (viewbox[2] - viewbox[0]) / 2.0
    traslacion_y = viewbox[1] - (viewbox[3] - viewbox[1]) / 2.0
    return [ (traslacion_x + vector_0) * escala, 
            (traslacion_y + vector_1) * escala , 
            vector[2] * escala, 
            vector[3] * escala,
            - vector[4]  ]
    #return [ vector_0 , vector_1 , vector[2], vector[3], - vector[4]  ]

In [ ]:
coordenadas2 = [ adaptar_coordenadas(x, viewbox1) for x in coordenadas1]
coordenadas2

In [ ]:
def construir_bloques(path_archivo = 'ChullaQuito.svg', nombre_capa = 'layer7', pendientes_anteriores = None, 
                      altura = 25 , espesor = 0.1 , escala = 5.0 / 2):
    data = []
    if not pendientes_anteriores is None:
        data = pendientes_anteriores
    else:
        data = importar_svg( nombre_capa = nombre_capa, path_archivo = path_archivo )
        viewbox = obtener_viewbox(path_archivo)
        # data = [ adaptar_coordenadas(x, viewbox, escala) for x in data]
    pendientes = []
    #rpa.init(visual_automation = True, chrome_browser = False)
    print('Iniciando secuencia');
    secuencia_activa = True
    for bloque in tqdm(data, desc="Construyendo Bloques..."):
        bloque_adaptado = adaptar_coordenadas(bloque, viewbox, escala)
        if secuencia_activa == False:
            pendientes.append(bloque)
            continue
        if not crear_caja( bloque_adaptado[0], bloque_adaptado[1], altura, bloque_adaptado[2], bloque_adaptado[3], espesor, bloque_adaptado[4] ):
            print('Secuencia interrumpida')
            pendientes.append(bloque)
            secuencia_activa = False
    print('Secuencia finalizada')
    #rpa.close()
    if len(pendientes) == 0:
        pendientes = None
    return pendientes_anteriores

In [ ]:
pendientes = None

In [ ]:
# Construir el piso
pendientes = construir_bloques(path_archivo = PATH_SVG, nombre_capa = LAYER_SVG, 
                               pendientes_anteriores = pendientes, altura = 8, espesor = 5, escala = 7.5 )
if not pendientes is None:
    print("quedan {0} pendientes".format(len(pendientes)))

In [ ]:
data = importar_svg( nombre_capa = LAYER_SVG, path_archivo = PATH_SVG )
data = [ [y for y in x ] + adaptar_coordenadas(x) for x in data ]
data = [ {'x':x[5], 'y':x[6], 'width':x[7], 'height':x[8], 'rotation':x[9], 'x1':x[0], 'y2':x[1], 'width1':x[2], 'height1':x[3], 'rotation1':x[4], } for x in data ]
##for x in data:
data = pd.DataFrame(data)
data
#vector_x = [[1,2], [4,5]]
#vector_y = [[4,5]]
#vector_z = [ x + [sum(x)] for x in vector_x ]
#vector_z

In [ ]:
adaptar_coordenadas([-62.876225, 99.632347, 0.2, 6.2879076,-40.217861])

In [ ]:
adaptar_coordenadas([-62.876225, 99.632347, 0.2, 6.2879076,0])

In [ ]:
print('{0} a {1}\n{2} a {3}'.format(data['x'].min(), data['x'].max(), data['y'].min(), data['y'].max())) 